# 📢📢📢 Projet NLP : La parole aux citoyens (Make.org - Le tourisme vert en Ille-et-Vilaine)📢📢📢
# Quels sont nos thèmes? Comment prioriser? Quel thematique aborder en premier?
## Bibliothèques nécessaires

In [1]:
import pandas as pd
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objects as go
import math
from plotly.colors import sample_colorscale

## Lecture des données

In [2]:
# Chargement des jeux de données
df = pd.read_parquet(
    "../data/processed/req_tourisme_responsable_final.parquet",
    engine='pyarrow'
)

df.head(3)

,content,agree_count,disagree_count,neutral_count,agree_score,disagree_score,neutral_score,doNotCare,doNotUnderstand,doable,...,embedding_prop,embedding_prop_x,embedding_prop_y,cluster_label,top_15_words,cluster_label_cah,top_15_words_cah,cluster_label_hdbscan,cluster_label_best,best_top_15_words
0,Il faut une surveillance des lieux touristique...,27,7,7,0.65,0.18,0.17,1,0,8,...,"[0.026475444436073303, 0.013547265902161598, 0...",60.746189,15.809195,9,"[dechet, lieux, tri, eau, poubelle, mettre, se...",4,"[camping, dechet, eau, lieux, car, poubelle, t...",-1,4,"[local, dechet, responsable, lieux, developper..."
1,Il faut donner des sacs poubelles et des cendr...,56,18,15,0.62,0.19,0.19,0,0,21,...,"[0.034178830683231354, 0.05750420689582825, 0....",54.815662,12.159612,7,"[camping, car, parking, lieux, aire, developpe...",4,"[camping, dechet, eau, lieux, car, poubelle, t...",-1,4,"[local, dechet, responsable, lieux, developper..."
2,Il faut récompenser et mettre en avant les act...,35,7,14,0.62,0.13,0.25,0,5,12,...,"[-0.011080465279519558, 0.053166456520557404, ...",-22.491800,47.501347,4,"[responsable, developper, rendre, lieux, acteu...",1,"[mettre, responsable, pouvoir, etre, creer, gr...",-1,3,"[responsable, durable, local, acteur, mettre, ..."


## Définition des thématiques
Pour aller assez vite, l'utilisation de la colonne des 15 mots les plus présents me semble intéressante. Pour aller plus loin, il pourrait y avoir des discussions avec les métiers du marketing sur le nombre de thématiques et leurs noms (regarder les phrases par échantillons, etc...)

In [3]:
# Définir la largeur d'affichage souhaitée (par exemple, 1000)
largeur_affichage = 2000

# Appliquer la nouvelle largeur d'affichage
pd.set_option('display.max_colwidth', largeur_affichage)

In [4]:
themes = df.groupby('cluster_label_best')['best_top_15_words'].first().reset_index()
themes

,cluster_label_best,best_top_15_words
0,0,"[limiter, naturel, site, acce, interdir, mettre, certain, place, espace, parc, camping, lieux, vehicule, taxer, nombre]"
1,1,"[transport, train, velo, developper, avion, commun, prix, vol, rendre, vacance, moins, voiture, accessible, proposer, moyen]"
2,2,"[cyclable, local, developper, piste, produire, itineraire, velo, voie, ville, favoriser, securisee, veloroute, promouvoir, transport, reseau]"
3,3,"[responsable, durable, local, acteur, mettre, creer, favoriser, lieux, site, developper, place, pouvoir, environnement, rendre, vacance]"
4,4,"[local, dechet, responsable, lieux, developper, mettre, vacance, territoire, environnement, proposer, creer, sensibiliser, eau, chaque, acteur]"


### Table de correspondance
- Cluster 0 : préserver les sites naturels et limiter les accès
- Cluster 1 : rendre les transports bas carbones plus attractifs et pénaliser les voyages trop émetteur en CO2 (voiture, avion)
- Cluster 2 : donner plus de pouvoir et de route pour les vélos, sécuriser les pistes
- Cluster 3 : responsabiliser les gens sur les enjeux environnementaux pour agir sur la durabilité
- Cluster 4 : sensibiliser les territoires pour une meilleure gestion de l'eau et des déchets

In [5]:
LabelDict = {
    0: "préserver les sites naturels et limiter les accès",
    1: "rendre les transports bas carbones plus attractifs et pénaliser les voyages trop émetteur en CO2 (voiture, avion)",
    2: "donner plus de pouvoir et de route pour les vélos, sécuriser les pistes",
    3: "responsabiliser les gens sur les enjeux environnementaux pour agir sur la durabilité",
    4: "sensibiliser les territoires pour une meilleure gestion de l'eau et des déchets"
}

df["cluster_label_best"] = df["cluster_label_best"].replace(LabelDict)

## Priorisation des thématiques : avec la taille des clusters

In [6]:
cluster_counts = df['cluster_label_best'].value_counts().reset_index()
cluster_counts.columns = ['cluster_label_best', 'count']

In [7]:
fig = px.pie(cluster_counts,
             values = 'count',
             names = 'cluster_label_best',
             color_discrete_sequence = px.colors.sequential.Greens,
             hole=0.3,
        )

# Enlever la légende
fig.update_layout(
    title=dict(
        text='Répartition des Clusters',
        font=dict(
            size=25,
            color="black",
            family="Arial"
        ),
        xanchor="center",
        yanchor="top",
        x=0.5,
        y=0.95
    ),
    template='simple_white',
    hoverlabel_font_size=16,
    showlegend=False
)
# Ajouter les labels des clusters sur le camembert
fig.update_traces(
    textposition='inside',
    textinfo='percent+value',
    marker_line=dict(color='black', width=2),
    hoverlabel_bgcolor="darkgreen",
    hovertemplate='<b>%{label}</b><br>Nombre de phrases: %{value}<extra></extra>'
)

fig.show()

In [8]:
# Sauvegarder le graphique en tant que fichier HTML
pyo.plot(fig, filename='../outputs/Répartition des clusters par taille.html', auto_open=False)

'../outputs/Répartition des clusters par taille.html'

### Premier ordre de priorisation :
- Action n°1 : sensibiliser les territoires pour une meilleure gestion de l'eau et des déchets
- Action n°2 : responsabiliser les gens sur les enjeux environnementaux pour agir sur la durabilité
- Action n°3 : rendre les transports bas carbones plus attractifs et pénaliser les voyages trop émetteur en CO2 (voiture, avion)
- Action n°4 : préserver les sites naturels et limiter les accès
- Action n°5 : donner plus de pouvoir et de route pour les vélos, sécuriser les pistes

## Deuxième idée de priorisation : quels clusters font le plus débat, provoque le plus d'émotions ?
Mix de plusieurs informations :
- Le nombre de points d'interrogation
- Le nombre de points d'exclamation
- Le décompte des votes (simplement le nombre de réactions totales)

In [9]:
reactions_list = [
    "agree_count", "disagree_count", "neutral_count",
    "doNotCare", "doNotUnderstand", "doable",
    'impossible', 'likeIt', 'noOpinion',
    'noWay', 'platitudeAgree', 'platitudeDisagree',
    'nb_interrogations', 'nb_exclamation'
]

In [10]:
cluster_reactions = df.groupby("cluster_label_best")[reactions_list].sum().sum(axis=1).reset_index()
cluster_reactions.columns = ['cluster_label_best', 'Nombre de réactions']

In [11]:
fig = px.pie(cluster_reactions,
             values = 'Nombre de réactions',
             names = 'cluster_label_best',
             color_discrete_sequence = px.colors.sequential.Greens,
             hole=0.3,
        )

# Enlever la légende
fig.update_layout(
    title=dict(
        text='Nombre de réactions des clusters',
        font=dict(
            size=25,
            color="black",
            family="Arial"
        ),
        xanchor="center",
        yanchor="top",
        x=0.5,
        y=0.95
    ),
    template='simple_white',
    hoverlabel_font_size=16,
    showlegend=False
)
# Ajouter les labels des clusters sur le camembert
fig.update_traces(
    textposition='inside',
    textinfo='percent+value',
    marker_line=dict(color='black', width=2),
    hoverlabel_bgcolor="darkgreen",
    hovertemplate='<b>%{label}</b><br>Nombre de réactions: %{value}<extra></extra>'
)

fig.show()

In [12]:
# Sauvegarder le graphique en tant que fichier HTML
pyo.plot(fig, filename='../outputs/Nombre de réactions des clusters.html', auto_open=False)

'../outputs/Nombre de réactions des clusters.html'

### Deuxième ordre de priorisation :
- Action n°1 : sensibiliser les territoires pour une meilleure gestion de l'eau et des déchets
- Action n°2 : rendre les transports bas carbones plus attractifs et pénaliser les voyages trop émetteur en CO2 (voiture, avion)
- Action n°3 : responsabiliser les gens sur les enjeux environnementaux pour agir sur la durabilité
- Action n°4 : préserver les sites naturels et limiter les accès
- Action n°5 : donner plus de pouvoir et de route pour les vélos, sécuriser les pistes

Ca ne change pas beaucoup pour une raison assez simple : le nombre de réactions et fatalement impacté par la taille des clusters

## Troisième idée de priorisation : même chose en prenant une moyenne de réactions
- Nombre total de réactions par cluster
- Dicision de ce nombre par la taille du cluster

In [13]:
# Fusionner les deux DataFrames
cluster_reactions = pd.merge(cluster_reactions, cluster_counts, on='cluster_label_best')

# Calculer la moyenne des réactions par élément dans chaque cluster
cluster_reactions['Moyenne des réactions'] = cluster_reactions['Nombre de réactions'] / cluster_reactions['count']

In [14]:
# Créer le graphique en camembert
fig = px.pie(cluster_reactions,
             values='Moyenne des réactions',
             names='cluster_label_best',
             color_discrete_sequence=px.colors.sequential.Greens,
             hole=0.3,
             )

# Enlever la légende et personnaliser le layout
fig.update_layout(
    title=dict(
        text='Moyennes des réactions des clusters (nombre totales des réactions / taille du cluster)',
        font=dict(
            size=25,
            color="black",
            family="Arial"
        ),
        xanchor="center",
        yanchor="top",
        x=0.5,
        y=0.95
    ),
    template='simple_white',
    hoverlabel_font_size=16,
    showlegend=False
)

# Ajouter les labels des clusters sur le camembert
fig.update_traces(
    textposition='inside',
    textinfo='percent+value',
    texttemplate='%{percent:.1%}<br>%{value:.0f}',
    marker_line=dict(color='black', width=2),
    hoverlabel_bgcolor="darkgreen",
    hovertemplate='<b>%{label}</b><br>Nombre de réactions: %{value:.0f}<extra></extra>'  # Formater les valeurs sans virgules
)

fig.show()

In [15]:
# Sauvegarder le graphique en tant que fichier HTML
pyo.plot(fig, filename='../outputs/Moyennes des réactions des clusters.html', auto_open=False)

'../outputs/Moyennes des réactions des clusters.html'

### Troisième ordre de priorisation :
- Action n°1 : donner plus de pouvoir et de route pour les vélos, sécuriser les pistes
- Action n°2 : sensibiliser les territoires pour une meilleure gestion de l'eau et des déchets
- Action n°3 : rendre les transports bas carbones plus attractifs et pénaliser les voyages trop émetteur en CO2 (voiture, avion)
- Action n°4 : préserver les sites naturels et limiter les accès
- Action n°5 : responsabiliser les gens sur les enjeux environnementaux pour agir sur la durabilité

La communauté des vélos n'est peut-être pas grande, mais elle fait du bruit!!

## Quatrième proposition : les clusters par consensus
- Rapport entre agree_count et le total pour avoir les pourcentages dans chaque cluster.
- Celui qui aura le meilleur rapport sera priorisé, les autres viendront ensuite.

In [16]:
reactions_list = [
    "agree_count", "disagree_count", "neutral_count"
]

In [17]:
cluster_consensus = df.groupby("cluster_label_best")[reactions_list].sum().sum(axis=1).reset_index()
cluster_consensus.columns = ['cluster_label_best', 'Nombre de réactions']

cluster_consensus_pt2 = df.groupby("cluster_label_best")["agree_count"].sum().reset_index()
cluster_consensus_pt2.columns = ['cluster_label_best', 'Nombre de agree']

cluster_consensus = pd.merge(cluster_consensus, cluster_consensus_pt2, on='cluster_label_best')
cluster_consensus["Rapport de réactions positives"] = cluster_consensus['Nombre de agree'] / cluster_consensus['Nombre de réactions']

# Tronquer les noms de clusters à 30 caractères
cluster_consensus['cluster_label_truncated'] = cluster_consensus['cluster_label_best'].apply(
    lambda x: x[:27] + '...' if len(x) > 30 else x
)

In [18]:
cols = 2  # Nombre de colonnes
n_clusters = len(cluster_consensus)
rows = math.ceil(n_clusters / cols)

padding = 0.1  # Espace horizontal et vertical (en pourcentage de cellule)

In [19]:
fig = go.Figure()

for i, (_, row) in enumerate(cluster_consensus.iterrows()):
    row_idx = i // cols
    col_idx = i % cols

    # Plage totale pour la cellule (sans padding)
    x0 = col_idx / cols
    x1 = (col_idx + 1) / cols
    y0 = 1 - (row_idx + 1) / rows
    y1 = 1 - row_idx / rows

    # Appliquer le padding pour créer une marge entre les jauges
    x_pad = (x1 - x0) * padding
    y_pad = (y1 - y0) * padding

    domain_x = [x0 + x_pad, x1 - x_pad]
    domain_y = [y0 + y_pad, y1 - y_pad]

    colorscale = "RdYlGn"  # rouge → jaune → vert
    value = row['Rapport de réactions positives']
    bar_color = sample_colorscale(colorscale, [value])[0]

    fig.add_trace(go.Indicator(
        mode="gauge+number",
        value=row['Rapport de réactions positives'],
        domain={'x': domain_x, 'y': domain_y},
        title={'text': row['cluster_label_truncated']},
        gauge={
            'axis': {'range': [0, 1]},
            'bar': {'color': bar_color},
            'steps': [
                {'range': [0, 0.5], 'color': "lightgray"},
                {'range': [0.5, 0.8], 'color': "gray"},
                {'range': [0.8, 1], 'color': "darkgray"}
            ],
        }
    ))

fig.update_layout(
    title=dict(
        text='Scores de satisfaction par cluster',
        font=dict(size=25, color="black", family="Arial"),
        x=0.5, xanchor="center"
    ),
    template='simple_white',
    height=300 * rows,
    margin=dict(t=100)
)

fig.show()

In [20]:
# Sauvegarder le graphique en tant que fichier HTML
pyo.plot(fig, filename='../outputs/Scores de satisfaction par cluster.html', auto_open=False)

'../outputs/Scores de satisfaction par cluster.html'

### Quatrième ordre de priorisation :
- Action n°1 : donner plus de pouvoir et de route pour les vélos, sécuriser les pistes
- Action n°2 : sensibiliser les territoires pour une meilleure gestion de l'eau et des déchets
- Action n°3 : responsabiliser les gens sur les enjeux environnementaux pour agir sur la durabilité
- Action n°4 : préserver les sites naturels et limiter les accès
- Action n°5 : rendre les transports bas carbones plus attractifs et pénaliser les voyages trop émetteur en CO2 (voiture, avion)

J'aime bien cette façon de prioriser. Lancer des première action qui font consensus et laisser place à la discussion pour d'éventuels ajustements.